In [1]:
from datetime import datetime
import os
from time_utils.gpst import dt2gpst, gpst2dt, gpst_week_number, gpst_week_day

import zipfile
import tarfile
import gzip
import shutil

from ftplib import FTP
from os.path import basename, dirname, join, exists
from os import makedirs

import matplotlib.pyplot as plt

from ftplib import FTP
from os.path import basename, dirname, join, exists
from os import makedirs

import numpy as np
from pandas import DataFrame

In [2]:
import zipfile
import tarfile

import gzip
import shutil

rarfile = None
try:
    import rarfile
except ImportError:
    msg = '`rarfile` package not found.  ".rar" file decompression will not be available'
    print(msg)
    #raise ImportError(msg)

from os.path import dirname, join, basename
import subprocess

`rarfile` package not found.  ".rar" file decompression will not be available


In [3]:
import zipfile
import tarfile

import gzip
import shutil

rarfile = None
try:
    import rarfile
except ImportError:
    msg = '`rarfile` package not found.  ".rar" file decompression will not be available'
    print(msg)
    #raise ImportError(msg)

from os.path import dirname, join, basename
import subprocess

from scipy.constants import c

`rarfile` package not found.  ".rar" file decompression will not be available


In [4]:
from RINEX2Helper import format_filepath, ftp_download, fix_bad_zip_file, decompress
from rinex2 import parse_RINEX2_obs_file

`rarfile` package not found.  ".rar" file decompression will not be available


In [5]:
for day in range(21, 28):
    ftp_host = 'cddis.gsfc.nasa.gov'
    filepath_template = \
        'pub/gps/data/daily/{yyyy}/{ddd}/{yy}o/nist{ddd}0.{yy}o.Z'
    url_filepath = format_filepath(filepath_template, datetime(2017, 8, day))
    filepath = os.path.join('/Users/liuzijun 1/Projects/gnss-research/data/rinex/', url_filepath)
    if not os.path.exists(os.path.dirname(filepath)):
        os.makedirs(os.path.dirname(filepath))
    if not os.path.exists(filepath):
        success = ftp_download(ftp_host, url_filepath, filepath)
    subprocess.call('uncompress -f ' + filepath.replace(' ', '\ '), shell=True)
    filepath = filepath[:-2]
    print (filepath)

/Users/liuzijun 1/Projects/gnss-research/data/rinex/pub/gps/data/daily/2017/233/17o/nist2330.17o
/Users/liuzijun 1/Projects/gnss-research/data/rinex/pub/gps/data/daily/2017/234/17o/nist2340.17o
/Users/liuzijun 1/Projects/gnss-research/data/rinex/pub/gps/data/daily/2017/235/17o/nist2350.17o
/Users/liuzijun 1/Projects/gnss-research/data/rinex/pub/gps/data/daily/2017/236/17o/nist2360.17o
/Users/liuzijun 1/Projects/gnss-research/data/rinex/pub/gps/data/daily/2017/237/17o/nist2370.17o
/Users/liuzijun 1/Projects/gnss-research/data/rinex/pub/gps/data/daily/2017/238/17o/nist2380.17o
/Users/liuzijun 1/Projects/gnss-research/data/rinex/pub/gps/data/daily/2017/239/17o/nist2390.17o


In [6]:
def download_and_decompress(station_id, dt, overwrite=False):
    ftp_host = 'cddis.gsfc.nasa.gov'
    filepath_template = 'pub/gps/data/daily/{yyyy}/{ddd}/{yy}o/{station_id}{ddd}0.{yy}o.Z'
    rinex_dir = '/Users/liuzijun 1/Projects/gnss-research/data/rinex/'
    url_filepath = format_filepath(filepath_template, dt, params={'station_id': station_id})
    filepath = os.path.join(rinex_dir, url_filepath)
    if not os.path.exists(os.path.dirname(filepath)):
        os.makedirs(os.path.dirname(filepath))
    if overwrite or (not os.path.exists(filepath) and not os.path.exists(filepath[:-2])):
        success = ftp_download(ftp_host, url_filepath, filepath)
    if overwrite or (os.path.exists(filepath) and not os.path.exists(filepath[:-2])):
        subprocess.call('uncompress -f ' + filepath.replace(' ', '\ '), shell=True)
    filepath = filepath[:-2]
    return filepath 

In [36]:
Number_of_Jump_Point = []
Name_of_Satellite = []
for day in range(21, 28):
    dt = datetime(2017, 8, day)
    station_id = 'nist'
    filepath = download_and_decompress(station_id, dt)
    header, observations = parse_RINEX2_obs_file(filepath)
    satellites = observations['satellites']
    time = observations['time']
    sat_ids = satellites.keys()
    
    for sat_id in sat_ids:
        header, observations = parse_RINEX2_obs_file(filepath)
        sat = observations['satellites'][sat_id]
        kappa_u = 40.308e16
        fL1 = 1.57542e9
        fL2 = 1.2276e9
        P1 = sat['L1']['pseudorange']
        P2 = sat['L2']['pseudorange']
        TEC = (P1 - P2) / (kappa_u * (1 / fL1**2 - 1 / fL2**2))

        L1 = sat['L1']['carrier'] * c / fL1
        L2 = sat['L2']['carrier'] * c / fL2
        TEC_rel = -(L1 - L2) / (kappa_u * (1 / fL1**2 - 1 / fL2**2))
    
        TEC_diff = np.diff(TEC_rel)
        threshold = 0.7
        indices = abs(TEC_diff) > threshold
        
        plot_filename = '{0:04}{1:02}{2:02}_{3}'.format(dt.year, dt.month, dt.day, sat_id)
        Name_of_Satellite.append(plot_filename)
        
        Number_of_Jump_Point.append(np.sum(indices))
        
Table_Satellite_NumberJumpPoint = {'Name_of_Satellite_Station': Name_of_Satellite, 
                                   'Number_of_Jump_Point': Number_of_Jump_Point}
df = DataFrame(Table_Satellite_NumberJumpPoint, columns = ['Name_of_Satellite_Station', 'Number_of_Jump_Point'])
export_csv = df.to_csv('export_table.csv', index = None, header = True)

ValueError: operands could not be broadcast together with shapes (966,) (965,) 